In [3]:
from IPython import get_ipython
from importlib import reload

# Import the desired libraries
import txgnn

# Reload the libraries
reload(txgnn)

from txgnn import TxData, TxGNN, TxEval
import wandb

/home/tysinger/.conda/envs/txgnn_env2/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
TxData_inst = TxData(data_folder_path = '/om/user/tysinger/kg/')
TxData_inst.prepare_split(split = 'random', seed = 42, no_kg = False)

Found local copy...
Found local copy...
Found local copy...
Found saved processed KG... Loading...
Splits detected... Loading splits....
Creating DGL graph....
Done!


In [7]:
sweep_configuration = {
            "method": "bayes",
            "name": "sweep",
            "metric": {"goal": "minimize", "name": "validation_loss"},
            "parameters": {
                "n_hid": {"values":[64,128,256,512,1280]}, 
                "n_inp": {"values":[64,128,256,512,1280]}, 
                "n_out": {"values":[64,128,256,512,1280]}, 
                "epochs": {"min": 10, "max": 300},
                "lr": {"max": 0.1, "min": 0.0001},
            },
        }

sweep_id = wandb.sweep(sweep_configuration)

Create sweep with ID: 8pqejm31
Sweep URL: https://wandb.ai/eptysinger/uncategorized/sweeps/8pqejm31


In [9]:
def train():
    TxGNN_model = txgnn.TxGNN(data = TxData_inst, 
            weight_bias_track = True,
            proj_name = 'MEng',
            exp_name = 'Sweep',
            device = 'cpu'
            )

    print('Initializing Model')
    TxGNN_model.model_initialize(n_hid = wandb.config.n_hid, 
                    n_inp = wandb.config.n_inp, 
                    n_out = wandb.config.n_out, 
                    proto = False, #made this False
                    proto_num = 3,
                    attention = False,
                    sim_measure = 'all_nodes_profile',
                    bert_measure = 'disease_name',
                    agg_measure = 'rarity',
                    num_walks = 200,
                    walk_mode = 'bit',
                    path_length = 2)
    
    TxGNN_model.load_pretrained('/om/user/tysinger/models/pretrain_random_1')
    print('Finetuning Model')
    TxGNN_model.finetune(n_epoch = wandb.config.epochs, 
                learning_rate = wandb.config.lr,
                train_print_per_n = 5,
                valid_per_n = 50,
                save_path = '/om/user/tysinger/models', 
                name = 'sweep')
    

In [10]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: 6b88usyd with config:
wandb: 	epochs: 263
wandb: 	lr: 0.07284665201381782
wandb: 	n_hid: 64
wandb: 	n_inp: 128
wandb: 	n_out: 256
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Initializing Model
Finetuning Model


Run 6b88usyd errored: DGLError('[16:59:43] /opt/dgl/src/array/cpu/./spmm_blocking_libxsmm.h:267: Failed to generate libxsmm kernel for the SpMM operation.To disable libxsmm, use dgl.use_libxsmm(false).\nStack trace:\n  [bt] (0) /home/tysinger/.conda/envs/txgnn_env2/lib/python3.8/site-packages/dgl/libdgl.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x4f) [0x2acf8b59cbff]\n  [bt] (1) /home/tysinger/.conda/envs/txgnn_env2/lib/python3.8/site-packages/dgl/libdgl.so(void dgl::aten::cpu::SpMMRedopCsrOpt<long, float, dgl::aten::cpu::op::CopyLhs<float>, dgl::aten::cpu::op::Add<float> >(dgl::BcastOff const&, dgl::aten::CSRMatrix const&, dgl::runtime::NDArray, dgl::runtime::NDArray, dgl::runtime::NDArray, dgl::runtime::NDArray, dgl::runtime::NDArray)+0x3d4) [0x2acf8b7e9e54]\n  [bt] (2) /home/tysinger/.conda/envs/txgnn_env2/lib/python3.8/site-packages/dgl/libdgl.so(void dgl::aten::cpu::SpMMSumCsrLibxsmm<long, float, dgl::aten::cpu::op::CopyLhs<float> >(dgl::BcastOff const&, dgl::aten::CSRMatrix co

Initializing Model
Finetuning Model


In [3]:

TxGNN_model = txgnn.TxGNN(data = TxData_inst, 
            weight_bias_track = True,
            proj_name = 'MEng',
            exp_name = 'Test',
            device = 'cpu'
            )



"""
TxGNN_model.model_initialize(n_hid = 1280, 
                    n_inp = 1280, 
                    n_out = 1280, 
                    proto = False, #made this False
                    proto_num = 3,
                    attention = False,
                    sim_measure = 'all_nodes_profile',
                    bert_measure = 'disease_name',
                    agg_measure = 'rarity',
                    num_walks = 200,
                    walk_mode = 'bit',
                    path_length = 2)
"""

# to load a pretrained model: 
TxGNN_model.load_pretrained('/om/user/tysinger/models/random_finetuned150')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: eptysinger. Use `wandb login --relogin` to force relogin


In [9]:
## here we did not run this, since the output is too long to fit into the notebook
TxGNN_model.pretrain(n_epoch = 1, 
               learning_rate = 1e-3,
               batch_size = 1024, 
               train_print_per_n = 20)

Creating minibatch pretraining dataloader...
Start pre-training with #param: 1015000


DGLError: [22:33:24] /opt/dgl/src/array/cpu/./spmm_blocking_libxsmm.h:267: Failed to generate libxsmm kernel for the SpMM operation.To disable libxsmm, use dgl.use_libxsmm(false).
Stack trace:
  [bt] (0) /home/tysinger/.conda/envs/txgnn_env2/lib/python3.8/site-packages/dgl/libdgl.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x4f) [0x2b9a0fd37bff]
  [bt] (1) /home/tysinger/.conda/envs/txgnn_env2/lib/python3.8/site-packages/dgl/libdgl.so(void dgl::aten::cpu::SpMMRedopCsrOpt<long, float, dgl::aten::cpu::op::CopyLhs<float>, dgl::aten::cpu::op::Add<float> >(dgl::BcastOff const&, dgl::aten::CSRMatrix const&, dgl::runtime::NDArray, dgl::runtime::NDArray, dgl::runtime::NDArray, dgl::runtime::NDArray, dgl::runtime::NDArray)+0x3d4) [0x2b9a0ff84e54]
  [bt] (2) /home/tysinger/.conda/envs/txgnn_env2/lib/python3.8/site-packages/dgl/libdgl.so(void dgl::aten::cpu::SpMMSumCsrLibxsmm<long, float, dgl::aten::cpu::op::CopyLhs<float> >(dgl::BcastOff const&, dgl::aten::CSRMatrix const&, dgl::runtime::NDArra

In [5]:
## here as a demo, the n_epoch is set to 30. Change it to n_epoch = 500 when you use it


TxGNN_model.finetune(n_epoch = 15, 
               learning_rate = 5e-4,
               train_print_per_n = 5,
               valid_per_n = 50,
               save_path = '/om/user/tysinger/models', 
                name = 'random_finetune')

DGLError: [12:08:36] /opt/dgl/src/array/cpu/./spmm_blocking_libxsmm.h:267: Failed to generate libxsmm kernel for the SpMM operation.To disable libxsmm, use dgl.use_libxsmm(false).
Stack trace:
  [bt] (0) /home/tysinger/.conda/envs/txgnn_env2/lib/python3.8/site-packages/dgl/libdgl.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x4f) [0x2b048e361bff]
  [bt] (1) /home/tysinger/.conda/envs/txgnn_env2/lib/python3.8/site-packages/dgl/libdgl.so(void dgl::aten::cpu::SpMMRedopCsrOpt<long, float, dgl::aten::cpu::op::CopyLhs<float>, dgl::aten::cpu::op::Add<float> >(dgl::BcastOff const&, dgl::aten::CSRMatrix const&, dgl::runtime::NDArray, dgl::runtime::NDArray, dgl::runtime::NDArray, dgl::runtime::NDArray, dgl::runtime::NDArray)+0x3d4) [0x2b048e5aee54]
  [bt] (2) /home/tysinger/.conda/envs/txgnn_env2/lib/python3.8/site-packages/dgl/libdgl.so(void dgl::aten::cpu::SpMMSumCsrLibxsmm<long, float, dgl::aten::cpu::op::CopyLhs<float> >(dgl::BcastOff const&, dgl::aten::CSRMatrix const&, dgl::runtime::NDArra

In [ ]:
TxGNN_model.save_model('./model_noproto')
#TxGNN_model.load_pretrained('./model_noproto')

In [ ]:
TxGNN_model.train_graphmask(relation = None, #changed from 'indication'
                      learning_rate = 3e-4,
                      allowance = 0.005,
                      epochs_per_layer = 1,
                      penalty_scaling = 1,
                      valid_per_n = 20)



In [ ]:
output = TxGNN_model.retrieve_save_gates('./model_noproto')
TxGNN_model.save_graphmask_model('./graphmask_model_noproto')

In [ ]:
from txgnn import TxEval
TxEval_model = TxEval(model = TxGNN_model)

In [ ]:
# evaluate individual diseases
#result = TxEval_model.eval_disease_centric(disease_idxs = [12661.0, 11318.0], 
                                     #relation = 'indication', 
                                     #save_result = False)

# evaluate the entire test set
result = TxEval_model.eval_molfunc_centric(molfunc_idxs = 'test_set', 
                                     show_plot = True, 
                                     verbose = True, 
                                     save_result = False,
                                     return_raw = False)

In [ ]:
TxEval_model.retrieve_disease_idxs_test_set('indication')